<a href="https://colab.research.google.com/github/Mechanics-Mechatronics-and-Robotics/ML-2025a/blob/main/PINN_Plates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [ ]:
# Install dependencies for Colab
!pip install pytorch-lightning clearml

import math
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from clearml import Task
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass

In [ ]:
#Enter your code here to implement Step 2 of the logging instruction as it is shown below
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=ZP02U03C6V5ER4K9VWRNZT7EWA5ZTV
%env CLEARML_API_SECRET_KEY=BtA5GXZufr6QGpaqhX1GSKPTvaCt56OLqaNqUGLNoxx2Ye8Ctwbui0Ln5OXVnzUgH4I

# 🧠 Theoretical Introduction: Dimensionless PINNs for Parallel Plate Flow

We consider the steady, fully developed flow of an incompressible Newtonian fluid between two infinite parallel plates separated by distance \(h\).  
The velocity is unidirectional along \(x\), varying only in the transverse direction \(y\), so the governing equation reduces to:

$$
\mu \frac{d^2 u}{dy^2} = \frac{dp}{dx}
$$

where $\mu$ is dynamic viscosity, $\frac{dp}{dx}$ — constant pressure gradient (typically negative).

The **no-slip boundary conditions** are applied at both plates:
<!--  -->
$$
u\left(-\tfrac{h}{2}\right) = 0, \quad u\left(\tfrac{h}{2}\right) = 0
$$

---

## ✅ Nondimensionalization

We define dimensionless variables:

$$
Y = \frac{y}{h}, \qquad U = \frac{u}{U_\mathrm{ref}}, \qquad
\Pi = \frac{h^2}{\mu U_\mathrm{ref}} \left(\frac{dp}{dx}\right)
$$

Substituting into the PDE gives the **dimensionless Poisson equation**:

$$
\frac{d^2 U}{dY^2} = \Pi
$$

with **boundary conditions**:

$$
U(-\tfrac{1}{2}) = 0, \quad U(\tfrac{1}{2}) = 0
$$

The **analytic solution** in nondimensional form is:

$$
U(Y) = \frac{\Pi}{2} \left( Y^2 - \tfrac{1}{4} \right)
$$

---

## ⚙️ Physics-Informed Neural Network (PINN) Formulation

We approximate $U(Y)$ using a neural network $\hat{U}(Y)$ and enforce physics via loss minimization.

The total loss combines:

- **PDE residual loss** — enforcing the governing equation  
- **Boundary loss** — enforcing no-slip at $Y = \pm \tfrac{1}{2}$

$$
\mathcal{L} =
\underbrace{
\frac{1}{N_r} \sum_{i=1}^{N_r}
\left|
\frac{d^2 \hat{U}}{dY^2}(Y_i) - \Pi
\right|^2
}_{\text{Physics loss}}
+
\underbrace{
\frac{1}{N_b} \sum_{j=1}^{N_b}
\left|
\hat{U}(Y_j) - U_j
\right|^2
}_{\text{Boundary loss}}
$$

# Config

In [ ]:
torch.set_default_dtype(torch.float32)
pl.seed_everything(42)

# ----------------------------
# Config
# ----------------------------
@dataclass
class Config:
    # Physics parameters for dimensionalization
    h: float = 1e-3                # channel height [m]
    mu: float = 1e-3               # dynamic viscosity [Pa·s]
    rho: float = 1e+3              # density [kg / m^3]
    dpdx: float = -1e+3        # pressure gradient [Pa/m]

    # training hyperparams
    lr: float = 1e-3
    epochs: int = 1000
    n_colloc_per_Y: int = 64       # spatial points per batch
    n_boundary_per_Y: int = 2      # boundary points (fixed: Y=-0.5, 0.5)
    batch_size_pi: int = 20         # number of Pi values per batch
    n_pi_total: int = 100          # total number of Pi values
    pi_min: float = 1e-0
    pi_max: float = 1e+1

    # data split ratios
    train_ratio: float = 0.6
    val_ratio: float = 0.2
    test_ratio: float = 0.2

    # network
    layers: tuple = (2, 64, 64, 64, 64, 1)

    # settings
    check_val_every_n_epoch: int = 100
    task_name: str = "PINN_plates_dimless_generalized"

cfg = Config()

# Dataset and Dataloaders

In [ ]:
# ----------------------------
# Dataset Classes
# ----------------------------
class PiSpatialDataset(Dataset):
    """Dataset that returns batches of spatial points for specific Pi values"""
    def __init__(self, pi_values, n_colloc_per_Y, n_boundary_per_Y, device):
        self.pi_values = pi_values
        self.n_colloc_per_Y = n_colloc_per_Y
        self.n_boundary_per_Y = n_boundary_per_Y
        self.device = device

        # Precompute spatial grids
        self.Y_colloc = torch.linspace(-0.5, 0.5, n_colloc_per_Y, device=device).reshape(-1, 1)
        self.Y_boundary = torch.tensor([[-0.5], [0.5]], device=device, dtype=torch.float32)

    def __len__(self):
        return len(self.pi_values)

    def __getitem__(self, idx):
        pi = self.pi_values[idx]

        # Collocation points
        Y_colloc_batch = self.Y_colloc
        Pi_colloc_batch = torch.full_like(Y_colloc_batch, pi)
        X_colloc = torch.cat([Y_colloc_batch, Pi_colloc_batch], dim=1)

        # Boundary points
        Y_boundary_batch = self.Y_boundary
        Pi_boundary_batch = torch.full_like(Y_boundary_batch, pi)
        X_boundary = torch.cat([Y_boundary_batch, Pi_boundary_batch], dim=1)
        U_boundary = torch.zeros_like(Y_boundary_batch)

        return X_colloc, X_boundary, U_boundary

class PiBatchDataModule(pl.LightningDataModule):
    def __init__(self, cfg: Config, device="cpu"):
        super().__init__()
        self.cfg = cfg
        self.device = device

    def setup(self, stage=None):
        # Generate Pi values
        n_total = self.cfg.n_pi_total
        self.pi_all = torch.linspace(
            self.cfg.pi_min, self.cfg.pi_max, n_total, device=self.device
        )

        # Split into train/val/test
        n_train = int(n_total * self.cfg.train_ratio)
        n_val = int(n_total * self.cfg.val_ratio)
        n_test = n_total - n_train - n_val

        # Shuffle indices
        indices = torch.randperm(n_total)
        train_idx = indices[:n_train]
        val_idx = indices[n_train:n_train + n_val]
        test_idx = indices[n_train + n_val:]

        self.pi_train = self.pi_all[train_idx]
        self.pi_val = self.pi_all[val_idx]
        self.pi_test = self.pi_all[test_idx]

        print(f"Dataset sizes - Train: {len(self.pi_train)}, Val: {len(self.pi_val)}, Test: {len(self.pi_test)}")

    def train_dataloader(self):
        dataset = PiSpatialDataset(
            self.pi_train,
            self.cfg.n_colloc_per_Y,
            self.cfg.n_boundary_per_Y,
            self.device
        )
        return DataLoader(dataset, batch_size=self.cfg.batch_size_pi,
                          num_workers=0, shuffle=True)

    def val_dataloader(self):
        dataset = PiSpatialDataset(
            self.pi_val,
            self.cfg.n_colloc_per_Y,
            self.cfg.n_boundary_per_Y,
            self.device
        )
        return DataLoader(dataset, batch_size=self.cfg.batch_size_pi,
                          num_workers=0, shuffle=False)

    def test_dataloader(self):
        dataset = PiSpatialDataset(
            self.pi_test,
            self.cfg.n_colloc_per_Y,
            self.cfg.n_boundary_per_Y,
            self.device
        )
        return DataLoader(dataset, batch_size=self.cfg.batch_size_pi, shuffle=False)

# Model

In [ ]:
task = Task.init(
    project_name="PINN_Project",   # choose a descriptive project name
    task_name=cfg.task_name,  # descriptive task name
)

# task.connect(params)
tb_logger = TensorBoardLogger(
    save_dir="logs",  # ClearML will automatically monitor this
    name=cfg.task_name,
    # version=f"bs_{HP['batch_size']}_lr_{HP['lr_simclr']}"
  )

# ----------------------------
# PINN Model
# ----------------------------
class PINN(pl.LightningModule):
    def __init__(self, layers, lr):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.net = self._build_mlp(layers)

    def _build_mlp(self, sizes):
        layers = []
        for i in range(len(sizes)-1):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if i < len(sizes)-2:
                layers.append(nn.Tanh())
        return nn.Sequential(*layers)

    def forward(self, X):
        return self.net(X)

    def _second_deriv_wrt_Y(self, X):
        X_temp = X.clone().requires_grad_(True)
        U = self.forward(X_temp)

        # First derivative wrt Y
        dU_dY = torch.autograd.grad(
            U, X_temp,
            grad_outputs=torch.ones_like(U),
            create_graph=True,
            retain_graph=True
        )[0][:, 0:1]

        # Second derivative wrt Y
        d2U_dY2 = torch.autograd.grad(
            dU_dY, X_temp,
            grad_outputs=torch.ones_like(dU_dY),
            create_graph=True,
            retain_graph=True
        )[0][:, 0:1]

        return d2U_dY2

    def training_step(self, batch, batch_idx):
        X_colloc, X_bc, U_bc = batch

        # Reshape batches: [batch_size_pi * n_points, 2]
        batch_size_pi = X_colloc.shape[0]
        n_colloc = X_colloc.shape[1]
        n_bc = X_bc.shape[1]

        X_colloc_flat = X_colloc.reshape(-1, 2)
        X_bc_flat = X_bc.reshape(-1, 2)
        U_bc_flat = U_bc.reshape(-1, 1)

        # PDE residual loss
        res = self._pde_residual(X_colloc_flat)
        pde_loss = torch.mean(res**2)
        # constraint = torch.mean(res)

        # Boundary loss
        U_pred_bc = self.forward(X_bc_flat)
        bc_loss = nn.functional.mse_loss(U_pred_bc, U_bc_flat)

        loss = pde_loss + bc_loss #+ constraint
        #  + 0.01 * physical_constraint_loss
        self.log_dict({
            "train_pde": pde_loss,
            "train_bc": bc_loss,
            # "train_pc": physical_constraint_loss,
            "train_loss": loss
        }, prog_bar=True)
        return loss

    def _pde_residual(self, X_colloc):
        U_YY = self._second_deriv_wrt_Y(X_colloc)
        Pi = X_colloc[:, 1:2]
        res = U_YY - Pi
        return res

    def validation_step(self, batch, batch_idx):
        X_colloc, X_bc, U_bc = batch

        # Flatten batches
        X_colloc_flat = X_colloc.reshape(-1, 2)
        X_bc_flat = X_bc.reshape(-1, 2)
        U_bc_flat = U_bc.reshape(-1, 1)

        with torch.enable_grad():
            res = self._pde_residual(X_colloc_flat)
            pde_loss = torch.mean(res ** 2)
            # constraint = torch.mean(res)
            U_pred_bc = self.forward(X_bc_flat)
            bc_loss = nn.functional.mse_loss(U_pred_bc, U_bc_flat)
            val_loss = pde_loss + bc_loss# + constraint

        self.log_dict({
            "val_pde": pde_loss,
            "val_bc": bc_loss,
            "val_loss": val_loss
        }, prog_bar=True)
        return val_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-5)

# ----------------------------
# Dimensionalization Utilities
# ----------------------------
def dimensionless_to_dimensional(U_dim, Y_dim, Pi, cfg):
    """Convert dimensionless results to dimensional"""
    # Velocity: u = U * U_ref
    U_ref = (cfg.h**2 * cfg.dpdx) / (cfg.mu * Pi)
    u_dimensional = U_dim * U_ref

    # Position: y = Y * h
    y_dimensional = Y_dim * cfg.h

    return u_dimensional, y_dimensional, cfg.dpdx

def analytical_solution_dimensional(y, dpdx, mu, h):
    """Analytical solution for dimensional velocity"""
    # u(y) = (1/(2μ)) * (dp/dx) * (y² - (h/2)²)
    return (1/(2*mu)) * dpdx * (y**2 - (h/2)**2)

# ----------------------------
# Training
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
dm = PiBatchDataModule(cfg, device=device)
dm.setup()

model = PINN(layers=cfg.layers, lr=cfg.lr)

ckpt_cb = ModelCheckpoint(
    dirpath="checkpoints",
    filename="pinn_best",
    monitor="val_loss",
    save_top_k=1,
    mode="min",
)

trainer = pl.Trainer(
    max_epochs=cfg.epochs,
    accelerator="auto",
    callbacks=[ckpt_cb],
    log_every_n_steps=50,
    check_val_every_n_epoch=cfg.check_val_every_n_epoch
)

trainer.fit(model, datamodule=dm)


In [ ]:
task.close()

In [ ]:
# ----------------------------
# Testing and Results
# ----------------------------
best_model = PINN.load_from_checkpoint(ckpt_cb.best_model_path, layers=cfg.layers, lr=cfg.lr)
best_model.eval()
best_model = best_model.to(device)

# Test on various Pi values
test_pi_values = np.linspace(cfg.pi_min, cfg.pi_max, 6)
Y_test = torch.linspace(-0.5, 0.5, 100, device=device).reshape(-1, 1)

# Results tables
dimensionless_results = []
dimensional_results = []
detailed_results = []

# Calculate U_ref for each Pi and Reynolds number
def calculate_reynolds(pi, cfg):
    """Calculate Reynolds number for given Pi"""
    # From Pi definition: Π = (h²/(μ·U_ref)) * (dp/dx)
    # So U_ref = (h² * |dp/dx|) / (μ * Π)  [taking absolute value for Re calculation]
    U_ref = (cfg.h**2 * cfg.dpdx) / (cfg.mu * pi)
    Re = (cfg.rho * abs(U_ref) * cfg.h) / cfg.mu
    return Re, U_ref

# Create comprehensive plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
colors = plt.cm.viridis(np.linspace(0, 1, len(test_pi_values)))

# Plot 1: Dimensionless velocity profiles
ax1 = axes[0, 0]
for i, pi in enumerate(test_pi_values):
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (Y_test.cpu().numpy().flatten()**2 - 0.25)

    # Solid line for PINN, crosses for analytical
    ax1.plot(Y_test.cpu().numpy(), U_pred_dimless, '-', color=colors[i], linewidth=2, label=f'PINN Π={pi}')
    ax1.plot(Y_test.cpu().numpy(), U_analytical_dimless, 'x', color=colors[i], markersize=4, label=f'Analytical Π={pi}')

ax1.set_xlabel('Y (Dimensionless)')
ax1.set_ylabel('U (Dimensionless)')
ax1.set_title('Dimensionless Velocity Profiles')
ax1.legend()
ax1.grid(True)

# Plot 2: Dimensional velocity profiles
ax2 = axes[0, 1]
for i, pi in enumerate(test_pi_values):
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    # Calculate U_ref for this Pi
    Re, U_ref = calculate_reynolds(pi, cfg)

    # Convert to dimensional
    u_pred_dimensional = U_pred_dimless * U_ref
    y_dimensional = Y_test.cpu().numpy().flatten() * cfg.h

    # Analytical solution in dimensional form
    u_analytical_dimensional = analytical_solution_dimensional(
        y_dimensional, cfg.dpdx, cfg.mu, cfg.h
    )

    ax2.plot(y_dimensional, u_pred_dimensional, '-', color=colors[i], linewidth=2, label=f'PINN Π={pi}')
    ax2.plot(y_dimensional, u_analytical_dimensional, 'x', color=colors[i], markersize=4, label=f'Analytical Π={pi}')

ax2.set_xlabel('y [m]')
ax2.set_ylabel('u [m/s]')
ax2.set_title('Dimensional Velocity Profiles')
ax2.legend()
ax2.grid(True)

# Plot 3: Absolute velocity error (dimensionless)
ax3 = axes[1, 0]
for i, pi in enumerate(test_pi_values):
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (Y_test.cpu().numpy().flatten()**2 - 0.25)

    absolute_error = np.abs(U_pred_dimless - U_analytical_dimless)

    ax3.plot(Y_test.cpu().numpy(), absolute_error, '-', color=colors[i], linewidth=2, label=f'Π={pi}')

ax3.set_xlabel('Y (Dimensionless)')
ax3.set_ylabel('Absolute Error (Dimensionless)')
ax3.set_title('Absolute Velocity Error (Dimensionless)')
ax3.legend()
ax3.grid(True)

# Plot 4: Error distribution across Pi values
ax4 = axes[1, 1]
max_errors = []
center_errors = []
for pi in test_pi_values:
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (0.25 - Y_test.cpu().numpy().flatten()**2)

    max_error = np.max(np.abs(U_pred_dimless - U_analytical_dimless))
    center_error = np.abs(U_pred_dimless[50] - U_analytical_dimless[50])

    max_errors.append(max_error)
    center_errors.append(center_error)

ax4.plot(test_pi_values, max_errors, 'o-', linewidth=2, label='Max Error')
ax4.plot(test_pi_values, center_errors, 's-', linewidth=2, label='Center Error')
ax4.set_xlabel('Π')
ax4.set_ylabel('Error (Dimensionless)')
ax4.set_title('Error vs Π')
ax4.legend()
ax4.grid(True)
ax4.set_yscale('log')

plt.tight_layout()
plt.show()

# Calculate detailed results with Reynolds numbers
print("\n" + "="*80)
print("COMPREHENSIVE TEST RESULTS")
print("="*80)

for pi in test_pi_values:
    Pi_test = torch.full_like(Y_test, pi)
    X_test = torch.cat([Y_test, Pi_test], dim=1)

    with torch.no_grad():
        U_pred_dimless = best_model(X_test).cpu().numpy().flatten()

    U_analytical_dimless = (pi/2.0) * (Y_test.cpu().numpy().flatten()**2 - 0.25)

    # Calculate Reynolds number and U_ref
    Re, U_ref = calculate_reynolds(pi, cfg)

    # Convert to dimensional
    u_pred_dimensional = U_pred_dimless * U_ref
    u_analytical_dimensional = U_analytical_dimless * U_ref
    y_dimensional = Y_test.cpu().numpy().flatten() * cfg.h

    # Errors
    mse_dimless = np.mean((U_pred_dimless - U_analytical_dimless)**2)
    max_error_dimless = np.max(np.abs(U_pred_dimless - U_analytical_dimless))

    mse_dimensional = np.mean((u_pred_dimensional - u_analytical_dimensional)**2)
    max_error_dimensional = np.max(np.abs(u_pred_dimensional - u_analytical_dimensional))

    # Maximum velocity (at center)
    max_u_pred = U_pred_dimless[50]  # Y=0
    max_u_analytical = U_analytical_dimless[50]
    max_u_error = abs(max_u_pred - max_u_analytical)

    max_u_pred_dimensional = u_pred_dimensional[50]
    max_u_analytical_dimensional = u_analytical_dimensional[50]
    max_u_error_dimensional = abs(max_u_pred_dimensional - max_u_analytical_dimensional)

    dimensionless_results.append({
        "Pi": pi,
        "MSE": mse_dimless,
        "Max_Error": max_error_dimless,
        "U_pred_center": max_u_pred,
        "U_analytical_center": max_u_analytical,
        "Error_center": max_u_error
    })

    dimensional_results.append({
        "Pi": pi,
        "U_ref [m/s]": U_ref,
        "Re": Re,
        "MSE [m²/s²]": mse_dimensional,
        "Max_Error [m/s]": max_error_dimensional,
        "u_pred_center [m/s]": max_u_pred_dimensional,
        "u_analytical_center [m/s]": max_u_analytical_dimensional,
        "Error_center [m/s]": max_u_error_dimensional
    })

    detailed_results.append({
        "#": len(detailed_results) + 1,
        "Pi": pi,
        "Re": f"{Re:.2f}",
        "Max_U_Pred": f"{max_u_pred:.6f}",
        "Max_U_Analytical": f"{max_u_analytical:.6f}",
        "Max_U_Error": f"{max_u_error:.6f}",
        "MSE": f"{mse_dimless:.6f}"
    })

# Print detailed table with Re numbers
print("\nDETAILED DIMENSIONLESS RESULTS WITH REYNOLDS NUMBERS")
print("="*80)
detailed_df = pd.DataFrame(detailed_results)
print(detailed_df.to_string(index=False))

# Print dimensionless results
print("\n" + "="*80)
print("DIMENSIONLESS RESULTS")
print("="*80)
df_dimensionless = pd.DataFrame(dimensionless_results)
pd.set_option("display.float_format", "{:.6e}".format)
print(df_dimensionless.to_string(index=False))

# Print dimensional results
print("\n" + "="*80)
print("DIMENSIONAL RESULTS")
print("="*80)
df_dimensional = pd.DataFrame(dimensional_results)
print(df_dimensional.to_string(index=False))

# Summary statistics
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(f"Training Pi range: {cfg.pi_min} to {cfg.pi_max}")
print(f"Test Pi values: {test_pi_values}")
print(f"Physical parameters: h={cfg.h} m, μ={cfg.mu} Pa·s, ρ={cfg.rho} kg/m³, dp/dx={cfg.dpdx} Pa/m")
print(f"\nDimensionless Max Error across all tests: {max(r['Max_Error'] for r in dimensionless_results):.2e}")
print(f"Dimensional Max Error across all tests: {max(r['Max_Error [m/s]'] for r in dimensional_results):.2e} m/s")

# Reynolds number analysis
print(f"\nReynolds Number Range: {min(float(r['Re']) for r in dimensional_results):.2f} to {max(float(r['Re']) for r in dimensional_results):.2f}")

# Test on validation and test sets
print("\n" + "="*80)
print("VALIDATION AND TEST SET PERFORMANCE")
print("="*80)

def evaluate_dataset(pi_values, dataset_name):
    total_mse = 0
    total_max_error = 0
    count = 0

    with torch.no_grad():
        for pi in pi_values[:10]:  # Evaluate on first 10 values
            Pi_test = torch.full_like(Y_test, pi.item())
            X_test = torch.cat([Y_test, Pi_test], dim=1)

            U_pred_dimless = best_model(X_test).cpu().numpy().flatten()
            U_analytical_dimless = (pi.item()/2.0) * (Y_test.cpu().numpy().flatten()**2 - 0.25)

            mse = np.mean((U_pred_dimless - U_analytical_dimless)**2)
            max_error = np.max(np.abs(U_pred_dimless - U_analytical_dimless))

            total_mse += mse
            total_max_error += max_error
            count += 1

    avg_mse = total_mse / count
    avg_max_error = total_max_error / count
    print(f"{dataset_name} set - Avg MSE: {avg_mse:.2e}, Avg Max Error: {avg_max_error:.2e}")

evaluate_dataset(dm.pi_train, "Train")
evaluate_dataset(dm.pi_val, "Validation")
evaluate_dataset(dm.pi_test, "Test")

# Additional analysis: Check if the model learned the physical relationship
print("\n" + "="*80)
print("PHYSICAL RELATIONSHIP ANALYSIS")
print("="*80)
print("Checking if U_max ∝ Π (should be linear relationship):")

# Extract max velocities vs Pi
pi_values_analysis = np.array([r['Pi'] for r in dimensionless_results])
max_u_values = np.array([r['U_pred_center'] for r in dimensionless_results])
max_u_analytical = np.array([r['U_analytical_center'] for r in dimensionless_results])

# Calculate linear regression for predicted values
slope_pred, intercept_pred = np.polyfit(pi_values_analysis, max_u_values, 1)
slope_analytical, intercept_analytical = np.polyfit(pi_values_analysis, max_u_analytical, 1)

print(f"Predicted relationship: U_max = {slope_pred:.6f} * Π + {intercept_pred:.6f}")
print(f"Analytical relationship: U_max = {slope_analytical:.6f} * Π + {intercept_analytical:.6f}")
print(f"Slope error: {abs(slope_pred - slope_analytical)/slope_analytical*100:.2f}%")

# Plot the relationship
plt.figure(figsize=(10, 6))
plt.plot(pi_values_analysis, max_u_values, 'bo-', label='PINN Prediction', linewidth=2)
plt.plot(pi_values_analysis, max_u_analytical, 'rx-', label='Analytical', linewidth=2)
plt.plot(pi_values_analysis, slope_pred * pi_values_analysis + intercept_pred, 'b--', alpha=0.5, label=f'PINN Fit: slope={slope_pred:.4f}')
plt.plot(pi_values_analysis, slope_analytical * pi_values_analysis + intercept_analytical, 'r--', alpha=0.5, label=f'Analytical: slope={slope_analytical:.4f}')
plt.xlabel('Π')
plt.ylabel('U_max (at Y=0)')
plt.title('Maximum Velocity vs Π (Checking Physical Relationship)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ----------------------------
# Batch Inspection Code
# ----------------------------
print("=" * 80)
print("BATCH STRUCTURE INSPECTION")
print("=" * 80)

# Get the first batch from train dataloader
train_dataloader = dm.train_dataloader()
first_batch = next(iter(train_dataloader))

print("Batch type:", type(first_batch))
print("Number of elements in batch:", len(first_batch))
print("\n")

# Unpack the batch
X_colloc, X_bc, U_bc = first_batch

print("1. COLLOCATION POINTS (X_colloc)")
print("   Shape:", X_colloc.shape)
print("   dtype:", X_colloc.dtype)
print("   device:", X_colloc.device)
print("\n   First 5 elements:")
print("   " + "=" * 50)
for i in range(min(5, X_colloc.shape[0])):
    print(f"   Batch element {i}:")
    print(f"     Shape: {X_colloc[i].shape}")
    print(f"     First 3 collocation points:")
    for j in range(min(3, X_colloc[i].shape[0])):
        y_val = X_colloc[i][j, 0].item()
        pi_val = X_colloc[i][j, 1].item()
        print(f"       Point {j}: Y = {y_val:8.4f}, Π = {pi_val:8.4f}")
    print()

print("\n2. BOUNDARY POINTS (X_bc)")
print("   Shape:", X_bc.shape)
print("   dtype:", X_bc.dtype)
print("   device:", X_bc.device)
print("\n   First 5 batch elements:")
print("   " + "=" * 50)
for i in range(min(5, X_bc.shape[0])):
    print(f"   Batch element {i}:")
    print(f"     Shape: {X_bc[i].shape}")
    print(f"     Boundary points:")
    for j in range(X_bc[i].shape[0]):
        y_val = X_bc[i][j, 0].item()
        pi_val = X_bc[i][j, 1].item()
        print(f"       Y = {y_val:8.4f}, Π = {pi_val:8.4f}")
    print()

print("\n3. BOUNDARY VALUES (U_bc)")
print("   Shape:", U_bc.shape)
print("   dtype:", U_bc.dtype)
print("   device:", U_bc.device)
print("\n   First 5 batch elements:")
print("   " + "=" * 50)
for i in range(min(5, U_bc.shape[0])):
    print(f"   Batch element {i}:")
    print(f"     Shape: {U_bc[i].shape}")
    print(f"     Boundary values (should all be 0):")
    for j in range(U_bc[i].shape[0]):
        u_val = U_bc[i][j, 0].item()
        print(f"       U = {u_val:8.4f}")
    print()

# Additional analysis: Check unique Pi values in this batch
print("\n4. BATCH ANALYSIS")
print("   " + "=" * 50)

# Extract all Pi values from collocation points
all_pi_values = []
for i in range(X_colloc.shape[0]):
    # All points in this batch element have the same Pi value
    pi_val = X_colloc[i][0, 1].item()
    all_pi_values.append(pi_val)

print(f"   Number of unique Π values in batch: {len(set(all_pi_values))}")
print(f"   Π values in this batch: {sorted(set(all_pi_values))}")
print(f"   Π range in batch: min={min(all_pi_values):.4f}, max={max(all_pi_values):.4f}")

# Check spatial distribution for first Pi value
first_pi = all_pi_values[0]
print(f"\n   Spatial distribution for Π = {first_pi:.4f}:")
first_batch_colloc = X_colloc[0]
y_values = first_batch_colloc[:, 0].cpu().numpy()
print(f"     Y range: min={y_values.min():.4f}, max={y_values.max():.4f}")
print(f"     Number of collocation points: {len(y_values)}")
print(f"     Y spacing: ~{(y_values.max() - y_values.min()) / len(y_values):.4f}")

# Verify boundary conditions
print(f"\n   Boundary condition verification for Π = {first_pi:.4f}:")
first_batch_bc = X_bc[0]
first_batch_u_bc = U_bc[0]
for j in range(first_batch_bc.shape[0]):
    y_val = first_batch_bc[j, 0].item()
    u_val = first_batch_u_bc[j, 0].item()
    print(f"     Y = {y_val:8.4f}, U = {u_val:8.4f} (should be 0)")

# Test the model on the first batch element
print("\n5. MODEL PREDICTION ON FIRST BATCH ELEMENT")
print("   " + "=" * 50)

with torch.no_grad():
    # Flatten the first batch element for prediction
    X_colloc_flat = X_colloc[0]  # Shape: [n_colloc_per_Y, 2]
    predictions = best_model(X_colloc_flat)

    print(f"   Input shape: {X_colloc_flat.shape}")
    print(f"   Output shape: {predictions.shape}")
    print(f"   Π value: {first_pi:.4f}")
    print(f"   Prediction range: min={predictions.min().item():.6f}, max={predictions.max().item():.6f}")
    print(f"   First 5 predictions:")
    for j in range(min(5, predictions.shape[0])):
        y_val = X_colloc_flat[j, 0].item()
        pred_val = predictions[j, 0].item()
        print(f"     Y = {y_val:8.4f}, U_pred = {pred_val:10.6f}")

# Compare with analytical solution for the first Pi value
print(f"\n6. COMPARISON WITH ANALYTICAL SOLUTION FOR Π = {first_pi:.4f}")
print("   " + "=" * 50)

analytical_solution = (first_pi/2.0) * (X_colloc_flat[:, 0].cpu().numpy()**2 - 0.25)

print("   First 5 points comparison:")
print("   " + "-" * 60)
print("      Y        U_PINN       U_Analytical     Error")
print("   " + "-" * 60)
for j in range(min(5, predictions.shape[0])):
    y_val = X_colloc_flat[j, 0].item()
    pred_val = predictions[j, 0].item()
    analytical_val = analytical_solution[j]
    error = abs(pred_val - analytical_val)
    print(f"   {y_val:8.4f}   {pred_val:12.6f}   {analytical_val:12.6f}   {error:12.6f}")

# Calculate overall error for this batch element
mse_batch = torch.mean((predictions.squeeze() - torch.tensor(analytical_solution, device=predictions.device))**2).item()
max_error_batch = torch.max(torch.abs(predictions.squeeze() - torch.tensor(analytical_solution, device=predictions.device))).item()

print(f"\n   Overall error for this batch element:")
print(f"     MSE: {mse_batch:.6e}")
print(f"     Max Error: {max_error_batch:.6e}")

print("\n" + "=" * 80)
print("BATCH INSPECTION COMPLETE")
print("=" * 80)